### Read in data file

In [1]:
datafile <- "data/export_chemistry.csv"
df <- read.csv(datafile,header=FALSE)

fields <- c("ID","FirstName","MiddleName","LastName","Institution")
columnNames <- c(
    paste("Trainee",fields,sep="."),
    paste("Mentor",fields,sep="."),
    "relationshipCode","relationshipType")

colnames(df) <- columnNames
head(df)

,Trainee.ID,Trainee.FirstName,Trainee.MiddleName,Trainee.LastName,Trainee.Institution,Mentor.ID,Mentor.FirstName,Mentor.MiddleName,Mentor.LastName,Mentor.Institution,relationshipCode,relationshipType
1,18761,Alan,P,Koretsky,National Institute of Neurological Disorders and Stroke,9,Melvin,P.,Klein,"University of California, Berkeley",1,student
2,53604,Jeroen,P.,van der Sluijs,Utrecht University,33687,JosÃ©,C.M.,van Eijndhoven,Utrecht University,1,student
3,20,John,A,Assad,Harvard Medical School,24,John,HR,Maunsell,University of Rochester,2,postdoc
4,12561,George,T.,Robillard,University of Groningen,52639,Philip,Edwin,Wilcox,"University of Washington, Seattle",1,student
5,189,Walter,Bradford,Cannon,Harvard Medical School,195,Henry,Pickering,Bowditch,Harvard Medical School,1,student
6,223,Carl,Friedrich Wilhelm,Ludwig,UniversitÃ¤t Leipzig,87777,Hermann,,Nasse,Philipps-UniversitÃ¤t Marburg,2,postdoc


In [18]:
### define root persion ID
root.ID <- unique(df[df$Mentor.FirstName == "Peter" & df$Mentor.LastName == "Schultz","Mentor.ID"])
stopifnot(length(root.ID) == 1)

#root.ID <- 62876 ### test on Virginia Cornish
#root.ID <- 63525 ### test on Hening Lin
    
root.ID


[1] 52763

In [19]:
getPerson <- function(ID, df) {
    p <- df[df$Trainee.ID==ID,c("Trainee.ID","Trainee.FirstName","Trainee.MiddleName","Trainee.LastName")]
    person <- list()
    person$ID <- as.character(p$Trainee.ID[1])
    person$FirstName <- as.character(p$Trainee.FirstName[1])
    person$MiddleName <- as.character(p$Trainee.MiddleName[1])
    person$LastName <- as.character(p$Trainee.LastName[1])
    person$name <- paste(c(
        substr(person$FirstName,0,1),
        substr(person$MiddleName,0,1),
        substr(person$LastName,0,1) ), collapse="")

    return(person) 
}

getChildren <- function(root.ID, df) {
    root <- getPerson(root.ID, df)
    children.df <- df[df$Mentor.ID == root$ID,]
#    children.ids <- unique(children.df$Trainee.ID)
#    print(paste("A: ", root$ID, children.ids))
#    print(children.df)

    children <- list()
    if( dim(children.df)[1] == 0 ) { return(children); }
#    for( child.id in children.ids ) {
    for( i in 1:dim(children.df)[1] ) {
       child.id <- children.df[i,"Trainee.ID"]
       child.type <- children.df[i,"relationshipType"]
#       print(paste("B: ", child.id))
       child <- getPerson(child.id, children.df)
       child$type <- child.type
#        print(child$ID)
       child$children <- getChildren( child$ID, df )
       children[[length(children)+1]] <- child
    }
    return(children)
}

root <- getPerson(root.ID, df)
root$children <- getChildren(root.ID, df)


In [31]:
library(jsonlite)
json <- jsonlite::toJSON(root,pretty=TRUE,auto_unbox=TRUE)
write(json, paste(c("output/output.json"),collapse=""))
